In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

from random import randrange
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

# ROOT_FOLDER = '../input/whale-categorization-playground'
ROOT_FOLDER = 'data'
TRAIN_FOLDER = ROOT_FOLDER + '/train/train/'
TEST_FOLDER = ROOT_FOLDER + '/test/test/'
WIDTH = 150
HEIGHT = 150

df_train = pd.read_csv(ROOT_FOLDER + '/train.csv')
df_test = pd.read_csv(ROOT_FOLDER + '/sample_submission.csv')
label_to_index = {}
index_to_label = {}

ROUNDS = 10
TRAIN_AMOUNT_PER_ROUND = len(df_train) * 10
OUT_OF_BAG_QNT = len(df_train) // 10
BATCH_SIZE = 1000

for index, (image, label) in df_train.iterrows():
    index_to_label[index] = label
    if not label in label_to_index:
        label_to_index[label] = len(label_to_index)

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=5, padding="same", activation="relu", input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding="valid"))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Conv2D(32, (15, 15), activation="relu"))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding="valid"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(4251, activation="softmax"))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])


# data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


# training
def next_index_in_bag(out_of_bag_indices):
    while True:
        idx = randrange(len(df_train))
        if not (idx in out_of_bag_indices):
            return idx

for r in range(ROUNDS):
    print('***** ROUND {} / {} *****'.format(r + 1, ROUNDS))
    out_of_bag = []
    for i in range(OUT_OF_BAG_QNT):
        out_of_bag.append(randrange(len(df_train)))
    
    for t in range(int(TRAIN_AMOUNT_PER_ROUND / BATCH_SIZE) + 1):
        print(str(t) + "/" + str(int(TRAIN_AMOUNT_PER_ROUND / BATCH_SIZE)))

        x_train = []
        y_train = []
        for i in range(BATCH_SIZE):
            idx = next_index_in_bag(out_of_bag)
            image = df_train.loc[idx]['Image']
            x_train.append(img_to_array(load_img(TRAIN_FOLDER + image, target_size=(WIDTH, HEIGHT))))
            y_train.append(np.zeros(4251))
            y_train[i][label_to_index[df_train.loc[idx]['Id']]] = 1

        x_train = np.array(x_train)
        y_train = np.array(y_train)

        datagen.fit(x_train)
        model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                            steps_per_epoch=x_train.shape[0] // BATCH_SIZE, verbose=False)
        model.fit(x_train, y_train, batch_size=BATCH_SIZE, verbose=False)
    
    # validation
    correct_preds = 0
    for idx in out_of_bag:
        image = df_train.loc[idx]['Image']
        x_test = np.array([img_to_array(load_img(TRAIN_FOLDER + image, target_size=(WIDTH, HEIGHT)))])
        pred = np.argmax(model.predict(x_test))
        
        if pred == label_to_index[df_train.loc[idx]['Id']]:
            correct_preds += 1
#         else:
#             print('predicted {}, but it was {}'.format(pred, label_to_index[df_train.loc[idx]['Id']]))
    print('- Round accuracy: {}'.format(correct_preds / OUT_OF_BAG_QNT))


# predicting
def sort_predictions(array):
    pairs = []
    for idx, v in enumerate(array):
        pairs.append((idx, v))
    pairs.sort(key=lambda p: p[1], reverse=True)
    
    top_sorted = []
    for p in pairs[:5]:
        top_sorted.append(index_to_label[p[0]])
#     print(pairs[:5], top_sorted)
    return top_sorted

print('----------------- PREDICTING -----------------')
predictions = []
for index, (image, labels) in df_test.iterrows():
    image = df_test.loc[index]['Image']
    x_test = np.array([img_to_array(load_img(TEST_FOLDER + image, target_size=(WIDTH, HEIGHT)))])
    preds = model.predict(x_test)[0]
    top_preds = sort_predictions(preds)
#     print(top_preds)
    predictions.append(top_preds)
    
    print(str(index) + '/' + str(len(df_test)))

# submission
submission = pd.DataFrame(data=[[df_test.loc[i]['Image'], "{} {} {} {} {}".format(*predictions[i])] for i in range(len(predictions))],
                           columns=['Image', 'Id'])
print(submission)
submission.to_csv('submission.csv', index=False)

***** ROUND 1 / 10 *****
0/98
1/98
2/98
3/98
4/98
5/98
6/98
7/98
8/98
9/98
10/98
11/98
12/98
13/98
14/98
15/98
16/98
17/98
18/98
19/98
20/98
21/98
22/98
23/98
24/98
25/98
26/98
27/98
28/98
29/98
30/98
31/98
32/98
33/98
34/98
35/98
36/98
37/98
38/98
39/98
40/98
41/98
42/98
43/98
44/98
45/98
46/98
47/98
48/98
49/98
50/98
51/98
52/98
53/98
54/98
55/98
56/98
57/98
58/98
59/98
60/98
61/98
62/98
63/98
64/98
65/98
66/98
67/98
68/98
69/98
70/98
71/98
72/98
73/98
74/98
75/98
76/98
77/98
78/98
79/98
80/98
81/98
82/98
83/98
84/98
85/98
86/98
87/98
88/98
89/98
90/98
91/98
92/98
93/98
94/98
95/98
96/98
97/98
98/98
- Round accuracy: 0.0883248730964467
***** ROUND 2 / 10 *****
0/98
1/98
2/98
3/98
4/98
5/98
6/98
7/98
8/98
9/98
10/98
11/98
12/98
13/98
14/98
15/98
16/98
17/98
18/98
19/98
20/98
21/98
22/98
23/98
24/98
25/98
26/98
27/98
28/98
29/98
30/98
31/98
32/98
33/98
34/98
35/98
36/98
37/98
38/98
39/98
40/98
41/98
42/98
43/98
44/98
45/98
46/98
47/98
48/98
49/98
50/98
51/98
52/98
53/98
54/98
55/98
56/

184/15610
185/15610
186/15610
187/15610
188/15610
189/15610
190/15610
191/15610
192/15610
193/15610
194/15610
195/15610
196/15610
197/15610
198/15610
199/15610
200/15610
201/15610
202/15610
203/15610
204/15610
205/15610
206/15610
207/15610
208/15610
209/15610
210/15610
211/15610
212/15610
213/15610
214/15610
215/15610
216/15610
217/15610
218/15610
219/15610
220/15610
221/15610
222/15610
223/15610
224/15610
225/15610
226/15610
227/15610
228/15610
229/15610
230/15610
231/15610
232/15610
233/15610
234/15610
235/15610
236/15610
237/15610
238/15610
239/15610
240/15610
241/15610
242/15610
243/15610
244/15610
245/15610
246/15610
247/15610
248/15610
249/15610
250/15610
251/15610
252/15610
253/15610
254/15610
255/15610
256/15610
257/15610
258/15610
259/15610
260/15610
261/15610
262/15610
263/15610
264/15610
265/15610
266/15610
267/15610
268/15610
269/15610
270/15610
271/15610
272/15610
273/15610
274/15610
275/15610
276/15610
277/15610
278/15610
279/15610
280/15610
281/15610
282/15610
283/15610


1012/15610
1013/15610
1014/15610
1015/15610
1016/15610
1017/15610
1018/15610
1019/15610
1020/15610
1021/15610
1022/15610
1023/15610
1024/15610
1025/15610
1026/15610
1027/15610
1028/15610
1029/15610
1030/15610
1031/15610
1032/15610
1033/15610
1034/15610
1035/15610
1036/15610
1037/15610
1038/15610
1039/15610
1040/15610
1041/15610
1042/15610
1043/15610
1044/15610
1045/15610
1046/15610
1047/15610
1048/15610
1049/15610
1050/15610
1051/15610
1052/15610
1053/15610
1054/15610
1055/15610
1056/15610
1057/15610
1058/15610
1059/15610
1060/15610
1061/15610
1062/15610
1063/15610
1064/15610
1065/15610
1066/15610
1067/15610
1068/15610
1069/15610
1070/15610
1071/15610
1072/15610
1073/15610
1074/15610
1075/15610
1076/15610
1077/15610
1078/15610
1079/15610
1080/15610
1081/15610
1082/15610
1083/15610
1084/15610
1085/15610
1086/15610
1087/15610
1088/15610
1089/15610
1090/15610
1091/15610
1092/15610
1093/15610
1094/15610
1095/15610
1096/15610
1097/15610
1098/15610
1099/15610
1100/15610
1101/15610
1102/15610

1760/15610
1761/15610
1762/15610
1763/15610
1764/15610
1765/15610
1766/15610
1767/15610
1768/15610
1769/15610
1770/15610
1771/15610
1772/15610
1773/15610
1774/15610
1775/15610
1776/15610
1777/15610
1778/15610
1779/15610
1780/15610
1781/15610
1782/15610
1783/15610
1784/15610
1785/15610
1786/15610
1787/15610
1788/15610
1789/15610
1790/15610
1791/15610
1792/15610
1793/15610
1794/15610
1795/15610
1796/15610
1797/15610
1798/15610
1799/15610
1800/15610
1801/15610
1802/15610
1803/15610
1804/15610
1805/15610
1806/15610
1807/15610
1808/15610
1809/15610
1810/15610
1811/15610
1812/15610
1813/15610
1814/15610
1815/15610
1816/15610
1817/15610
1818/15610
1819/15610
1820/15610
1821/15610
1822/15610
1823/15610
1824/15610
1825/15610
1826/15610
1827/15610
1828/15610
1829/15610
1830/15610
1831/15610
1832/15610
1833/15610
1834/15610
1835/15610
1836/15610
1837/15610
1838/15610
1839/15610
1840/15610
1841/15610
1842/15610
1843/15610
1844/15610
1845/15610
1846/15610
1847/15610
1848/15610
1849/15610
1850/15610

2507/15610
2508/15610
2509/15610
2510/15610
2511/15610
2512/15610
2513/15610
2514/15610
2515/15610
2516/15610
2517/15610
2518/15610
2519/15610
2520/15610
2521/15610
2522/15610
2523/15610
2524/15610
2525/15610
2526/15610
2527/15610
2528/15610
2529/15610
2530/15610
2531/15610
2532/15610
2533/15610
2534/15610
2535/15610
2536/15610
2537/15610
2538/15610
2539/15610
2540/15610
2541/15610
2542/15610
2543/15610
2544/15610
2545/15610
2546/15610
2547/15610
2548/15610
2549/15610
2550/15610
2551/15610
2552/15610
2553/15610
2554/15610
2555/15610
2556/15610
2557/15610
2558/15610
2559/15610
2560/15610
2561/15610
2562/15610
2563/15610
2564/15610
2565/15610
2566/15610
2567/15610
2568/15610
2569/15610
2570/15610
2571/15610
2572/15610
2573/15610
2574/15610
2575/15610
2576/15610
2577/15610
2578/15610
2579/15610
2580/15610
2581/15610
2582/15610
2583/15610
2584/15610
2585/15610
2586/15610
2587/15610
2588/15610
2589/15610
2590/15610
2591/15610
2592/15610
2593/15610
2594/15610
2595/15610
2596/15610
2597/15610

...

12752/15610
12753/15610
12754/15610
12755/15610
12756/15610
12757/15610
12758/15610
12759/15610
12760/15610
12761/15610
12762/15610
12763/15610
12764/15610
12765/15610
12766/15610
12767/15610
12768/15610
12769/15610
12770/15610
12771/15610
12772/15610
12773/15610
12774/15610
12775/15610
12776/15610
12777/15610
12778/15610
12779/15610
12780/15610
12781/15610
12782/15610
12783/15610
12784/15610
12785/15610
12786/15610
12787/15610
12788/15610
12789/15610
12790/15610
12791/15610
12792/15610
12793/15610
12794/15610
12795/15610
12796/15610
12797/15610
12798/15610
12799/15610
12800/15610
12801/15610
12802/15610
12803/15610
12804/15610
12805/15610
12806/15610
12807/15610
12808/15610
12809/15610
12810/15610
12811/15610
12812/15610
12813/15610
12814/15610
12815/15610
12816/15610
12817/15610
12818/15610
12819/15610
12820/15610
12821/15610
12822/15610
12823/15610
12824/15610
12825/15610
12826/15610
12827/15610
12828/15610
12829/15610
12830/15610
12831/15610
12832/15610
12833/15610
12834/15610
1283

13434/15610
13435/15610
13436/15610
13437/15610
13438/15610
13439/15610
13440/15610
13441/15610
13442/15610
13443/15610
13444/15610
13445/15610
13446/15610
13447/15610
13448/15610
13449/15610
13450/15610
13451/15610
13452/15610
13453/15610
13454/15610
13455/15610
13456/15610
13457/15610
13458/15610
13459/15610
13460/15610
13461/15610
13462/15610
13463/15610
13464/15610
13465/15610
13466/15610
13467/15610
13468/15610
13469/15610
13470/15610
13471/15610
13472/15610
13473/15610
13474/15610
13475/15610
13476/15610
13477/15610
13478/15610
13479/15610
13480/15610
13481/15610
13482/15610
13483/15610
13484/15610
13485/15610
13486/15610
13487/15610
13488/15610
13489/15610
13490/15610
13491/15610
13492/15610
13493/15610
13494/15610
13495/15610
13496/15610
13497/15610
13498/15610
13499/15610
13500/15610
13501/15610
13502/15610
13503/15610
13504/15610
13505/15610
13506/15610
13507/15610
13508/15610
13509/15610
13510/15610
13511/15610
13512/15610
13513/15610
13514/15610
13515/15610
13516/15610
1351

14116/15610
14117/15610
14118/15610
14119/15610
14120/15610
14121/15610
14122/15610
14123/15610
14124/15610
14125/15610
14126/15610
14127/15610
14128/15610
14129/15610
14130/15610
14131/15610
14132/15610
14133/15610
14134/15610
14135/15610
14136/15610
14137/15610
14138/15610
14139/15610
14140/15610
14141/15610
14142/15610
14143/15610
14144/15610
14145/15610
14146/15610
14147/15610
14148/15610
14149/15610
14150/15610
14151/15610
14152/15610
14153/15610
14154/15610
14155/15610
14156/15610
14157/15610
14158/15610
14159/15610
14160/15610
14161/15610
14162/15610
14163/15610
14164/15610
14165/15610
14166/15610
14167/15610
14168/15610
14169/15610
14170/15610
14171/15610
14172/15610
14173/15610
14174/15610
14175/15610
14176/15610
14177/15610
14178/15610
14179/15610
14180/15610
14181/15610
14182/15610
14183/15610
14184/15610
14185/15610
14186/15610
14187/15610
14188/15610
14189/15610
14190/15610
14191/15610
14192/15610
14193/15610
14194/15610
14195/15610
14196/15610
14197/15610
14198/15610
1419

14801/15610
14802/15610
14803/15610
14804/15610
14805/15610
14806/15610
14807/15610
14808/15610
14809/15610
14810/15610
14811/15610
14812/15610
14813/15610
14814/15610
14815/15610
14816/15610
14817/15610
14818/15610
14819/15610
14820/15610
14821/15610
14822/15610
14823/15610
14824/15610
14825/15610
14826/15610
14827/15610
14828/15610
14829/15610
14830/15610
14831/15610
14832/15610
14833/15610
14834/15610
14835/15610
14836/15610
14837/15610
14838/15610
14839/15610
14840/15610
14841/15610
14842/15610
14843/15610
14844/15610
14845/15610
14846/15610
14847/15610
14848/15610
14849/15610
14850/15610
14851/15610
14852/15610
14853/15610
14854/15610
14855/15610
14856/15610
14857/15610
14858/15610
14859/15610
14860/15610
14861/15610
14862/15610
14863/15610
14864/15610
14865/15610
14866/15610
14867/15610
14868/15610
14869/15610
14870/15610
14871/15610
14872/15610
14873/15610
14874/15610
14875/15610
14876/15610
14877/15610
14878/15610
14879/15610
14880/15610
14881/15610
14882/15610
14883/15610
1488

15486/15610
15487/15610
15488/15610
15489/15610
15490/15610
15491/15610
15492/15610
15493/15610
15494/15610
15495/15610
15496/15610
15497/15610
15498/15610
15499/15610
15500/15610
15501/15610
15502/15610
15503/15610
15504/15610
15505/15610
15506/15610
15507/15610
15508/15610
15509/15610
15510/15610
15511/15610
15512/15610
15513/15610
15514/15610
15515/15610
15516/15610
15517/15610
15518/15610
15519/15610
15520/15610
15521/15610
15522/15610
15523/15610
15524/15610
15525/15610
15526/15610
15527/15610
15528/15610
15529/15610
15530/15610
15531/15610
15532/15610
15533/15610
15534/15610
15535/15610
15536/15610
15537/15610
15538/15610
15539/15610
15540/15610
15541/15610
15542/15610
15543/15610
15544/15610
15545/15610
15546/15610
15547/15610
15548/15610
15549/15610
15550/15610
15551/15610
15552/15610
15553/15610
15554/15610
15555/15610
15556/15610
15557/15610
15558/15610
15559/15610
15560/15610
15561/15610
15562/15610
15563/15610
15564/15610
15565/15610
15566/15610
15567/15610
15568/15610
1556